In [2]:
from pyspark import SparkConf,SparkContext
from pyspark.storagelevel import StorageLevel

conf=SparkConf().setAppName('spark5').setMaster('local[*]')
sc=SparkContext(conf=conf)

22/05/12 12:37:39 WARN Utils: Your hostname, MaQiang resolves to a loopback address: 127.0.1.1; using 192.168.1.110 instead (on interface wlp2s0)
22/05/12 12:37:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/12 12:37:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## 广播变量 : 提前将本地list  --> RDD

广播变量,提交给执行者,节约内存和通讯时间

### (当本地list不大时候 广播效率比 RDD->RDD快)

In [8]:
l=[(1,'qiang',11),(2,'ma',13),(3,"tiantian",15),(4,'hushuop',67)]

In [9]:
name_list=sc.broadcast(l)   #提前声明为广播变量

In [10]:
rdd=sc.parallelize([(1,'math',99),(2,'fisica',40),(3,'english',67),(4,'python',788),(3,'read',467)])

In [11]:
def fun(x):
    name=''
    for i in name_list.value:  #name.vaule 可以当作使用list
        if x[0]==i[0]:
            name=i[1]
    return (name,x[1],x[2])

In [12]:
rdd.map(fun).collect()

[('qiang', 'math', 99),
 ('ma', 'fisica', 40),
 ('tiantian', 'english', 67),
 ('hushuop', 'python', 788),
 ('tiantian', 'read', 467)]

## 累加器

In [21]:
rdd= sc.parallelize([1,2,3,4,5,6,7,8,9,10],2)
count= 0 
def fun(x):
    global count
    count+=1
    print(count)
    
rdd.map(fun).collect()
print(count)   #输出是0,因为 count传给执行者,在分区内累加,不会传给driver ,应该是10

0


1
2
3
4
5
1
2
3
4
5


In [23]:
# 提前声明为累加器变量,结果是10
rdd= sc.parallelize([1,2,3,4,5,6,7,8,9,10],2)
count= sc.accumulator(0)
def fun(x):
    global count
    count+=1
    print(count)
    
rdd.map(fun).collect()
print(count)   #输出是10

10


1
2
3
4
5
1
2
3
4
5


# DAG图  sparkUI查看

宽依赖 : 父亲有多个子  (shuffuer)
窄依赖 : 其他情况

一个stage内部都是 ''' 窄依赖 '''

## spark 并行度

cpu核心的2-10倍数,一般不要改全局并行度(分区数)
### 不让CPU空闲